# Optimization - February 2018

## Data

### Demand

In [1]:
import pandas as pd
import numpy as np
import csv
demand = pd.read_csv('Feb 2018 Weekly Demand.csv')
demand2 = pd.read_csv('Feb 2018 Weekly Demand.csv')
demand2['Inventory Date'] = pd.to_datetime(demand2['Inventory Date']).astype('datetime64[D]')
demand = demand.fillna(0)
demand['Inventory Date'] = pd.to_datetime(demand['Inventory Date']).astype('datetime64[D]')
#
demand['ItemCode'] = demand['ItemCode'].str.lstrip('0')
demand2['ItemCode'] = demand2['ItemCode'].str.lstrip('0')
#
demand = demand.set_index([demand['Inventory Date'],demand['ItemCode'],demand['Branch']])
demand = demand.drop(['Inventory Date','ItemCode','Branch'], axis = 1)
demand = demand.T
columnlist = []
for c in demand.columns:
    x, y, z = c
    x = np.datetime64(x)
    x = x.astype('datetime64[D]')
    if y[0:2] == 'SR':
        y = y[2:]
        c2 = (x,y,z)
    else:
        c2 = (x,y,z)
    columnlist.append(c2)
demand.columns = columnlist
demand = demand.groupby(by = demand.columns,axis=1).sum()
# Example selection: 
# demand[weeks[1],'271','Toronto F&S']['Weekly Demand']

In [2]:
demand

,"(2018-02-04, 0200600, Toronto F&S)","(2018-02-04, 0240600, Toronto F&S)","(2018-02-04, 0241000, Toronto F&S)","(2018-02-04, 0241200, Toronto F&S)","(2018-02-04, 0241250, Toronto F&S)","(2018-02-04, 0241600, Edmonton F&S)","(2018-02-04, 0241600, Toronto F&S)","(2018-02-04, 0241601, Toronto F&S)","(2018-02-04, 0241608, Toronto F&S)","(2018-02-04, 0241616, Toronto F&S)",...,"(2018-02-25, Y412XWB, Baltimore F&S)","(2018-02-25, Y412XWB, Edmonton F&S)","(2018-02-25, Y412XWB, Fontana)","(2018-02-25, Y412XWB, Great Lakes CC)","(2018-02-25, Y412XWB, Houston F&S)","(2018-02-25, Y412XWB, Linden)","(2018-02-25, Y412XWB, Sacramento Main)","(2018-02-25, Y412XWB, Seattle F&S)","(2018-02-25, Y412XWB, Tampa F&S)","(2018-02-25, Y412XWB, Vancouver F&S)"
Weekly Demand,1,18,60,866,5146,16,1526,80,686,60,...,26,9,27,56,73,16,34,1,8,14


### Branch-to-Branch Cost

In [3]:
# costs = pd.read_csv('RandomCosts.csv')
# costs['Cost'].apply(lambda x: float(x))
# #
# costs['Item'] = costs['Item'].str.lstrip('0')
# #
# costs = costs.set_index(['From','To','Item'])
# costs = costs.T.to_dict()
# # Example of selection: 
# #costs['Atlanta F&S','Atlanta F&S','200600']['Cost']

In [3]:
regression={'Mileage':0.3328,'Weight':0.09408,'OLinden':- 99.8,'OBaltimore F&S':179.3,'OAtlanta F&S':258.7,'OTampa F&S':111.0,
           'O43302':325.1,'OGreat Lakes CC':335.3,'OHouston F&S':245.2,'OFontana':- 55.1,'OSacramento Main':- 218.4,'OSeattle F&S':38.9,'OToronto F&S':532.6,
           'DLinden':-17.8,'DBaltimore F&S':59.1,'DAtlanta F&S':-138.2,'DTampa F&S':107.7,
           'D43302':190.2,'DGreat Lakes CC':- 5.6,'DHouston F&S':- 348.5,'DFontana':431.5,'DSacramento Main':558.7,'DSeattle F&S':635.2,'DToronto F&S':407.6,
           'MM':- 0.000060,'WW':- 0.000002,'MW':0.000041}

import csv
reader = csv.reader(open('weights.csv', 'r',encoding = 'utf8'))
weights = {}
for row in reader:
    x, item,y, weight = row
    weights[item] = weight

miles = {}

with open("distance_df.csv", "r") as infile:
    reader = csv.reader(infile)
    headers = next(reader)[1:]
    for row in reader:
        miles[row[0]] = {key: float(value) for key, value in zip(headers, row[1:])}
def cost(i,j,k):
    jx=j
    kx=k
    if i not in weights.keys():
        if '0'+i in weights.keys():
            i='0'+i
        
        elif 'SR'+i in weights.keys():
            i='SR'+i
            
        else:
            print(i)
            weights[i]=1
    if jx in ['Edmonton F&S','Vancouver F&S','Montreal F&S']:
        jx='Toronto F&S'
    if kx in ['Edmonton F&S','Vancouver F&S','Montreal F&S']:
        kx='Toronto F&S'
    c=(float(weights[i])/40000)*(-286.0 + 0.3328*miles[k][j]+regression['O'+jx]+regression['D'+kx]+0.09408*float(40000)- 0.000060*(miles[k][j])**2 - 0.000002*float(40000)**2+0.000041*float(40000)*miles[k][j])
    return c

### Total Units in System

In [4]:
totaldata = pd.read_csv('Feb4Inv.csv')
#
nosr = []
totaldata['ItemCode'] = totaldata['ItemCode'].str.lstrip('0')
for c in totaldata['ItemCode']:
    if c[0:2] == 'SR':
        c2 = c[2:]
    else:
        c2 = c
    nosr.append(c2)
totaldata['ItemCode'] = nosr    
totaldata = totaldata.groupby(by = 'ItemCode').sum()
# Example selection: totaldata['TotalUnits']['4004']

### Dealing with Missing Data

In [5]:
w = pd.read_csv('FebDiscrepanciesNoSR.csv')
w.set_index('Item Code')
bad = w['Item Code']
bad = bad.tolist()
bad

['1579',
 '1579',
 '1579',
 '1579',
 '2053',
 '2053',
 '2053',
 '2053',
 '3721',
 '5764',
 '737',
 '225010',
 '225010',
 '225010',
 '225013',
 '225013',
 '225013',
 'VFTP463087',
 'VFTP463087',
 'VFTP463087',
 'VFTP463087',
 '7018',
 '7018',
 '7018',
 '7018',
 'CRCWSJ',
 '1718024',
 '1718024',
 '1718024',
 '2197',
 '2197',
 '2197',
 '2197',
 '2199',
 '2199',
 '2199',
 '2199',
 'AL1S',
 'AL1S',
 '6681',
 '2190',
 '2190',
 '2190',
 '2190',
 'PSH9140',
 'PSH9140',
 'WPP21008',
 'WPP21008',
 'WPP21012',
 'WPP21012',
 'VFTP552147',
 'VFTP552147',
 'VFTP552147',
 'VFTP552147',
 '2149',
 '2151',
 '2152',
 'BR20L',
 'CGRGTE',
 'CGRGTE',
 'SU5IR',
 'SU5IR',
 '6356',
 '6356',
 '6356',
 '5764',
 '3575',
 '3575',
 '3721',
 'B10',
 'B10',
 'B10',
 'B10',
 '10795',
 '2197',
 '2197',
 '2197',
 '2197',
 'SLB6',
 'SLB6',
 'SLB6',
 '2250142',
 '2250142',
 '2250143',
 '2250143',
 '1759',
 '1759',
 '1003',
 '1003',
 '1134',
 '1134',
 '8730',
 '8730',
 'B4',
 'B4',
 'CRCWSJ',
 '1579',
 '2053',
 '3721',
 '5

## Gurobi Model

In [6]:
from gurobipy import GRB, Model, quicksum
m = Model('AlumaFeb')

Academic license - for non-commercial use only


In [7]:
plist = demand2['ItemCode'].unique()
partlist=[]
for p in plist:
    if p[0:2] == 'SR':
        p2 = p[2:]
    else:
        p2 = p
    partlist.append(p2.lstrip('0'))
partlist = pd.Series(partlist)
partlist = partlist.unique()
partlist = list(filter(lambda a: a not in bad,partlist))

In [8]:
branchlist = demand2['Branch'].unique()
branchlist = pd.Series(branchlist)
branchlist = branchlist.sort_values()
weeks = demand2['Inventory Date'].unique()
weeks = weeks.astype('datetime64[D]')

In [9]:
indexlist = []
for part in partlist:
    for frombranch in branchlist:
        for tobranch in branchlist:
            for week in weeks:
                indexlist.append((part,frombranch,tobranch,week))
indexlist2 = []
for part in partlist:
    for branch in branchlist:
        for week in weeks:
            indexlist2.append((part,branch,week))

In [10]:
x = m.addVars(indexlist, name ='x')
y = m.addVars(indexlist2, name = "y")

### Constraining y values when t = 1
***

In [15]:
y1 = pd.read_csv('Feb4Inv.csv')
y1['InventoryDate'] = weeks[0]
nosr2 = []
for g in y1['ItemCode']:
    g = str(g)
    if g[0:2] == 'SR':
        g2 = g[2:]
    else:
        g2 = g
    nosr2.append(g2)
y1['ItemCode'] = nosr2 
y1 = y1[y1.Branch != 'USD Region Offices']
y1 = y1.groupby(by = ['ItemCode','Branch','InventoryDate']).sum()
# Example Selection: y1['InventoryQuantity']['11','Atlanta F&S', weeks[0]]
for idx,r in y1.iterrows():
    if idx in indexlist2:
        try:
            m.addConstr(y[idx[0],idx[1],weeks[0]] == np.maximum(r, demand[weeks[0],idx[0],idx[1]]['Weekly Demand'])) #y1['InventoryQuantity'][idx[0],idx[1],weeks[0]])
        except KeyError:
            m.addConstr(y[idx[0],idx[1],weeks[0]] == r)      
    else:
        pass
                               

***

In [16]:
for i in partlist:
    for j in branchlist:
        for t in weeks:
            m.addConstrs(x[i,j,k,t] == 0 for k in branchlist if k == j)
            try:
                m.addConstr(y[i,j,t] >= demand[t,i,j]['Weekly Demand'])
            except KeyError:
                pass
            m.addConstr(y[i,j,t] >= 0)
        for t in weeks[0:len(weeks)-1]:
            try:
                m.addConstr(y[i,j,t+np.timedelta64(7,'D')] == y[i,j,t] + 
                            quicksum(x[i,k,j,t] for k in branchlist) - 
                            quicksum(x[i,j,k,t+np.timedelta64(7,'D')] for k in branchlist))
            except KeyError:
                pass
           
for i in partlist:
        for t in weeks:
            try:
                m.addConstr(quicksum(y[i,g,t] for g in branchlist) + quicksum(x[i,q,w,t] for q in branchlist for w in branchlist) <= totaldata['TotalUnits'][i])
            except KeyError:
                pass

In [17]:
m.setObjective(quicksum(cost(i,j,k)*
                        x[i,j,k,t]
                        for i in partlist
                        for j in branchlist
                        for k in branchlist
                        for t in weeks),
               GRB.MINIMIZE)

In [18]:
m.optimize()
#m.computeIIS()
#m.write("model.ilp")

Optimize a model with 811764 rows, 1877512 columns and 6243138 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-05, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+05]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.153975e+07   0.000000e+00      2s
   10247    2.2353012e+04   0.000000e+00   0.000000e+00      4s

Solved in 10247 iterations and 3.97 seconds
Optimal objective  2.235301198e+04


In [19]:
status_code = {1:'LOADED', 2:'OPTIMAL', 3:'INFEASIBLE', 4:'INF_OR_UNBD', 5:'UNBOUNDED'}
status = m.status
               
print('The optimization status is {}'.format(status_code[status]))
if status == 2:  
     print('Optimal solution:')
     print('Optimal objective value:\n{}'.format(m.objVal))
     for v in m.getVars():
            if v.x != 0:
                print('%s = %g' % (v.varName, v.x))
     

The optimization status is OPTIMAL
Optimal solution:
Optimal objective value:
22353.01198148405
x[115,Linden,Montreal F&S,2018-02-04] = 61
x[115,Seattle F&S,Vancouver F&S,2018-02-04] = 6
x[137,Linden,Montreal F&S,2018-02-04] = 14
x[14,Sacramento Main,Fontana,2018-02-04] = 189
x[194,Atlanta F&S,Tampa F&S,2018-02-04] = 21
x[194,Sacramento Main,Fontana,2018-02-04] = 6
x[207,Atlanta F&S,Tampa F&S,2018-02-04] = 1
x[3902,Atlanta F&S,Tampa F&S,2018-02-04] = 115
x[3985,Fontana,Sacramento Main,2018-02-04] = 280
x[3985,Sacramento Main,Fontana,2018-02-04] = 113
x[3995,Atlanta F&S,Tampa F&S,2018-02-04] = 574
x[3995,Sacramento Main,Fontana,2018-02-04] = 28
x[3997,Sacramento Main,Fontana,2018-02-04] = 183
x[4004,Sacramento Main,Fontana,2018-02-04] = 56
x[4005,Atlanta F&S,Houston F&S,2018-02-04] = 1921
x[4005,Sacramento Main,Fontana,2018-02-04] = 78
x[4045,Sacramento Main,Fontana,2018-02-04] = 140
x[4046,Sacramento Main,Fontana,2018-02-04] = 744
x[4121,Atlanta F&S,Tampa F&S,2018-02-04] = 1627
x[4121,

x[1718010,Sacramento Main,Fontana,2018-02-04] = 14
x[1870031,Sacramento Main,Fontana,2018-02-04] = 137
x[1870040,Sacramento Main,Fontana,2018-02-04] = 106
x[1870050,Sacramento Main,Fontana,2018-02-04] = 232
x[1870060,Sacramento Main,Fontana,2018-02-04] = 1717
x[1870080,Sacramento Main,Fontana,2018-02-04] = 174
x[1870096,Sacramento Main,Fontana,2018-02-04] = 105
x[1870099,Sacramento Main,Fontana,2018-02-04] = 34
x[1870105,Sacramento Main,Fontana,2018-02-04] = 22
x[1870150,Sacramento Main,Fontana,2018-02-04] = 67
x[1870155,Sacramento Main,Fontana,2018-02-04] = 534
x[1870400,Sacramento Main,Fontana,2018-02-04] = 567
x[1870405,Sacramento Main,Fontana,2018-02-04] = 23
x[1870440,Sacramento Main,Fontana,2018-02-04] = 1549
x[1997151,Atlanta F&S,Tampa F&S,2018-02-04] = 150
x[201,Linden,Montreal F&S,2018-02-04] = 27
x[203E,Sacramento Main,Fontana,2018-02-04] = 105
x[2196,Fontana,Sacramento Main,2018-02-04] = 10
x[2200,Sacramento Main,Fontana,2018-02-04] = 7
x[2200,Seattle F&S,Vancouver F&S,2018-

y[16,Montreal F&S,2018-02-04] = 3832
y[16,Montreal F&S,2018-02-11] = 3832
y[16,Montreal F&S,2018-02-18] = 3832
y[16,Montreal F&S,2018-02-25] = 3832
y[16,Sacramento Main,2018-02-04] = 381
y[16,Sacramento Main,2018-02-11] = 381
y[16,Sacramento Main,2018-02-18] = 381
y[16,Sacramento Main,2018-02-25] = 381
y[16,Seattle F&S,2018-02-04] = 3705
y[16,Seattle F&S,2018-02-11] = 3705
y[16,Seattle F&S,2018-02-18] = 3705
y[16,Seattle F&S,2018-02-25] = 3705
y[16,Tampa F&S,2018-02-04] = 8227
y[16,Tampa F&S,2018-02-11] = 8227
y[16,Tampa F&S,2018-02-18] = 8227
y[16,Tampa F&S,2018-02-25] = 8227
y[16,Toronto F&S,2018-02-04] = 65729
y[16,Toronto F&S,2018-02-11] = 65729
y[16,Toronto F&S,2018-02-18] = 65729
y[16,Toronto F&S,2018-02-25] = 65729
y[16,Vancouver F&S,2018-02-04] = 1735
y[16,Vancouver F&S,2018-02-11] = 1735
y[16,Vancouver F&S,2018-02-18] = 1735
y[16,Vancouver F&S,2018-02-25] = 1735
y[1604,Atlanta F&S,2018-02-04] = 60
y[1604,Atlanta F&S,2018-02-11] = 60
y[1604,Atlanta F&S,2018-02-18] = 60
y[1604,A

y[2189,Montreal F&S,2018-02-25] = 337
y[2189,Sacramento Main,2018-02-04] = 9
y[2189,Sacramento Main,2018-02-11] = 9
y[2189,Sacramento Main,2018-02-18] = 9
y[2189,Sacramento Main,2018-02-25] = 9
y[2189,Tampa F&S,2018-02-04] = 4935
y[2189,Tampa F&S,2018-02-11] = 4935
y[2189,Tampa F&S,2018-02-18] = 4935
y[2189,Tampa F&S,2018-02-25] = 4935
y[2189,Toronto F&S,2018-02-04] = 3145
y[2189,Toronto F&S,2018-02-11] = 3145
y[2189,Toronto F&S,2018-02-18] = 3145
y[2189,Toronto F&S,2018-02-25] = 3145
y[232,Atlanta F&S,2018-02-04] = 2566
y[232,Atlanta F&S,2018-02-11] = 2566
y[232,Atlanta F&S,2018-02-18] = 2566
y[232,Atlanta F&S,2018-02-25] = 2566
y[232,Edmonton F&S,2018-02-04] = 162
y[232,Edmonton F&S,2018-02-11] = 162
y[232,Edmonton F&S,2018-02-18] = 162
y[232,Edmonton F&S,2018-02-25] = 162
y[232,Fontana,2018-02-04] = 203
y[232,Fontana,2018-02-11] = 203
y[232,Fontana,2018-02-18] = 203
y[232,Fontana,2018-02-25] = 203
y[232,Great Lakes CC,2018-02-04] = 4
y[232,Great Lakes CC,2018-02-11] = 4
y[232,Great 

y[4060,Houston F&S,2018-02-25] = 41346
y[4060,Seattle F&S,2018-02-04] = 370
y[4060,Seattle F&S,2018-02-11] = 370
y[4060,Seattle F&S,2018-02-18] = 370
y[4060,Seattle F&S,2018-02-25] = 370
y[4060,Tampa F&S,2018-02-04] = 300
y[4060,Tampa F&S,2018-02-11] = 300
y[4060,Tampa F&S,2018-02-18] = 300
y[4060,Tampa F&S,2018-02-25] = 300
y[4070,Atlanta F&S,2018-02-04] = 61
y[4070,Atlanta F&S,2018-02-11] = 61
y[4070,Atlanta F&S,2018-02-18] = 61
y[4070,Atlanta F&S,2018-02-25] = 61
y[4070,Houston F&S,2018-02-04] = 144
y[4070,Houston F&S,2018-02-11] = 144
y[4070,Houston F&S,2018-02-18] = 144
y[4070,Houston F&S,2018-02-25] = 144
y[4074,Atlanta F&S,2018-02-04] = 68
y[4074,Atlanta F&S,2018-02-11] = 68
y[4074,Atlanta F&S,2018-02-18] = 68
y[4074,Atlanta F&S,2018-02-25] = 68
y[4074,Houston F&S,2018-02-04] = 19
y[4074,Houston F&S,2018-02-11] = 19
y[4074,Houston F&S,2018-02-18] = 19
y[4074,Houston F&S,2018-02-25] = 19
y[4074,Tampa F&S,2018-02-04] = 10
y[4074,Tampa F&S,2018-02-11] = 10
y[4074,Tampa F&S,2018-02-

...

y[700,Seattle F&S,2018-02-11] = 17
y[700,Seattle F&S,2018-02-18] = 17
y[700,Seattle F&S,2018-02-25] = 17
y[700,Tampa F&S,2018-02-04] = 8
y[700,Tampa F&S,2018-02-11] = 8
y[700,Tampa F&S,2018-02-18] = 8
y[700,Tampa F&S,2018-02-25] = 8
y[732,Seattle F&S,2018-02-04] = 84
y[732,Seattle F&S,2018-02-11] = 84
y[732,Seattle F&S,2018-02-18] = 84
y[732,Seattle F&S,2018-02-25] = 84
y[7964,Seattle F&S,2018-02-04] = 4
y[7964,Seattle F&S,2018-02-11] = 4
y[7964,Seattle F&S,2018-02-18] = 4
y[7964,Seattle F&S,2018-02-25] = 4
y[8144,Fontana,2018-02-04] = 120
y[8144,Fontana,2018-02-11] = 120
y[8144,Fontana,2018-02-18] = 120
y[8144,Fontana,2018-02-25] = 120
y[8144,Seattle F&S,2018-02-04] = 888
y[8144,Seattle F&S,2018-02-11] = 888
y[8144,Seattle F&S,2018-02-18] = 888
y[8144,Seattle F&S,2018-02-25] = 888
y[8144,Vancouver F&S,2018-02-04] = 21
y[8144,Vancouver F&S,2018-02-11] = 21
y[8144,Vancouver F&S,2018-02-18] = 21
y[8144,Vancouver F&S,2018-02-25] = 21
y[89,Atlanta F&S,2018-02-04] = 2
y[89,Atlanta F&S,2018-

y[7931,Seattle F&S,2018-02-11] = 2
y[7931,Seattle F&S,2018-02-18] = 2
y[7931,Seattle F&S,2018-02-25] = 2
y[7931,Tampa F&S,2018-02-04] = 2
y[7931,Tampa F&S,2018-02-11] = 2
y[7931,Tampa F&S,2018-02-18] = 2
y[7931,Tampa F&S,2018-02-25] = 2
y[7931,Toronto F&S,2018-02-04] = 2
y[7931,Toronto F&S,2018-02-11] = 2
y[7931,Toronto F&S,2018-02-18] = 2
y[7931,Toronto F&S,2018-02-25] = 2
y[7991,Fontana,2018-02-04] = 21
y[7991,Fontana,2018-02-11] = 21
y[7991,Fontana,2018-02-18] = 21
y[7991,Fontana,2018-02-25] = 21
y[7991,Tampa F&S,2018-02-04] = 24
y[7991,Tampa F&S,2018-02-11] = 24
y[7991,Tampa F&S,2018-02-18] = 24
y[7991,Tampa F&S,2018-02-25] = 24
y[824,Atlanta F&S,2018-02-04] = 29
y[824,Atlanta F&S,2018-02-11] = 29
y[824,Atlanta F&S,2018-02-18] = 29
y[824,Atlanta F&S,2018-02-25] = 29
y[824,Houston F&S,2018-02-04] = 8
y[824,Houston F&S,2018-02-11] = 8
y[824,Houston F&S,2018-02-18] = 8
y[824,Houston F&S,2018-02-25] = 8
y[824,Tampa F&S,2018-02-04] = 35
y[824,Tampa F&S,2018-02-11] = 35
y[824,Tampa F&S,2

y[10237,Toronto F&S,2018-02-04] = 37
y[10237,Toronto F&S,2018-02-11] = 37
y[10237,Toronto F&S,2018-02-18] = 37
y[10237,Toronto F&S,2018-02-25] = 37
y[10345,Houston F&S,2018-02-04] = 4
y[10345,Houston F&S,2018-02-11] = 4
y[10345,Houston F&S,2018-02-18] = 4
y[10345,Houston F&S,2018-02-25] = 4
y[10345,Toronto F&S,2018-02-04] = 8
y[10345,Toronto F&S,2018-02-11] = 8
y[10345,Toronto F&S,2018-02-18] = 8
y[10345,Toronto F&S,2018-02-25] = 8
y[10386,Montreal F&S,2018-02-04] = 148
y[10386,Montreal F&S,2018-02-11] = 148
y[10386,Montreal F&S,2018-02-18] = 148
y[10386,Montreal F&S,2018-02-25] = 148
y[10386,Toronto F&S,2018-02-04] = 1588
y[10386,Toronto F&S,2018-02-11] = 1588
y[10386,Toronto F&S,2018-02-18] = 1588
y[10386,Toronto F&S,2018-02-25] = 1588
y[10386,Vancouver F&S,2018-02-04] = 149
y[10386,Vancouver F&S,2018-02-11] = 149
y[10386,Vancouver F&S,2018-02-18] = 149
y[10386,Vancouver F&S,2018-02-25] = 149
y[1043,Toronto F&S,2018-02-04] = 286
y[1043,Toronto F&S,2018-02-11] = 286
y[1043,Toronto F&S

y[1990845,Montreal F&S,2018-02-04] = 129
y[1990845,Montreal F&S,2018-02-11] = 129
y[1990845,Montreal F&S,2018-02-18] = 129
y[1990845,Montreal F&S,2018-02-25] = 129
y[1990845,Toronto F&S,2018-02-04] = 501
y[1990845,Toronto F&S,2018-02-11] = 501
y[1990845,Toronto F&S,2018-02-18] = 501
y[1990845,Toronto F&S,2018-02-25] = 501
y[1991100,Toronto F&S,2018-02-04] = 1105
y[1991100,Toronto F&S,2018-02-11] = 1105
y[1991100,Toronto F&S,2018-02-18] = 1105
y[1991100,Toronto F&S,2018-02-25] = 1105
y[1991105,Toronto F&S,2018-02-04] = 2088
y[1991105,Toronto F&S,2018-02-11] = 2088
y[1991105,Toronto F&S,2018-02-18] = 2088
y[1991105,Toronto F&S,2018-02-25] = 2088
y[1991110,Toronto F&S,2018-02-04] = 755
y[1991110,Toronto F&S,2018-02-11] = 755
y[1991110,Toronto F&S,2018-02-18] = 755
y[1991110,Toronto F&S,2018-02-25] = 755
y[1991120,Toronto F&S,2018-02-04] = 20
y[1991120,Toronto F&S,2018-02-11] = 20
y[1991120,Toronto F&S,2018-02-18] = 20
y[1991120,Toronto F&S,2018-02-25] = 20
y[1991200,Montreal F&S,2018-02-0

y[29,Fontana,2018-02-25] = 8
y[29,Toronto F&S,2018-02-04] = 176
y[29,Toronto F&S,2018-02-11] = 176
y[29,Toronto F&S,2018-02-18] = 176
y[29,Toronto F&S,2018-02-25] = 176
y[30,Fontana,2018-02-04] = 4
y[30,Fontana,2018-02-11] = 4
y[30,Fontana,2018-02-18] = 4
y[30,Fontana,2018-02-25] = 4
y[30,Toronto F&S,2018-02-04] = 415
y[30,Toronto F&S,2018-02-11] = 415
y[30,Toronto F&S,2018-02-18] = 415
y[30,Toronto F&S,2018-02-25] = 415
y[SLVP411,Edmonton F&S,2018-02-04] = 2
y[SLVP411,Edmonton F&S,2018-02-11] = 2
y[SLVP411,Edmonton F&S,2018-02-18] = 2
y[SLVP411,Edmonton F&S,2018-02-25] = 2
y[SLVP411,Houston F&S,2018-02-04] = 16
y[SLVP411,Houston F&S,2018-02-11] = 16
y[SLVP411,Houston F&S,2018-02-18] = 16
y[SLVP411,Houston F&S,2018-02-25] = 16
y[SLVP411,Montreal F&S,2018-02-04] = 26
y[SLVP411,Montreal F&S,2018-02-11] = 26
y[SLVP411,Montreal F&S,2018-02-18] = 26
y[SLVP411,Montreal F&S,2018-02-25] = 26
y[SLVP411,Toronto F&S,2018-02-04] = 52
y[SLVP411,Toronto F&S,2018-02-11] = 52
y[SLVP411,Toronto F&S,201